In [1]:
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/My Drive/Projects/recommendation/

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/Projects/recommendation


In [2]:
from dataset.dataset import SparseMatrixDataset
import torch
from torch.sparse import mm as smm
from tqdm.auto import tqdm
from multiprocessing import Pool
from functools import partial
from numba import jit
import gc

In [3]:
class config:
    user_path= "data/gowalla/user_list.txt"
    item_path= "data/gowalla/item_list.txt"
    edge_path= "data/gowalla/train.txt"

In [4]:
dataset = SparseMatrixDataset(config)

In [5]:
edges = dataset.edges
mat = smm(edges.t(), edges).coalesce()

In [6]:
I = mat.indices()
V = mat.values()

In [7]:
row, col = I
edge_mask = row.new_empty(row.size(0), dtype=torch.bool)
node_mask = row.new_empty(mat.size(0), dtype=torch.bool)

In [11]:
%%time
start_index = 0
batch_size = 200
end_index = mat.size(0)

diag = []
while start_index < end_index:
    if start_index + batch_size > end_index:
        batch_size = end_index - start_index
    node_mask.fill_(False)
    node_mask[start_index:start_index+batch_size] = True
    torch.index_select(node_mask, 0, row, out=edge_mask)
    tmpI = I[:, edge_mask]
    tmpI[0,:] -= start_index
    tmpV = V[edge_mask]
    tmpM = torch.sparse_coo_tensor(tmpI, tmpV, (batch_size, end_index)).to_dense().int()
    # topk_d += val.sum(dim=1)
    diag.append(torch.tensor([tmpM[idx, idx + start_index] for idx in range(batch_size)]))

    start_index += batch_size

CPU times: user 1min 43s, sys: 873 ms, total: 1min 44s
Wall time: 1min 43s


In [12]:
torch.arange(16).view(4,4).topk(2, dim=1)

torch.return_types.topk(values=tensor([[ 3,  2],
        [ 7,  6],
        [11, 10],
        [15, 14]]), indices=tensor([[3, 2],
        [3, 2],
        [3, 2],
        [3, 2]]))

In [ ]:
%%time
size = 100
d = [mat[i,i] for i in range(size)];

In [ ]:
def topk_diagonal_sparse(mat, k):
    d = []
    length = mat.size(0)
    for idx in tqdm(range(length)):
        row = mat[idx]
        
        if torch.sparse.sum(row) == 0: 
            d.append(0)
        else:
            row = row.to_dense()
            d.append(row.topk(k).values.sum())
        
    i = torch.stack([torch.arange(len(d)), torch.arange(len(d))], dim=0)
    v = torch.tensor(d)
    return torch.sparse_coo_tensor(i, v, mat.size())

def topk_similar_sparse(mat, k):
    i1 = []
    i2 = []
    vs = []
    for idx, row in enumerate(mat):
        if torch.sparse.sum(row) == 0: continue
        _, i = row.to_dense().topk(k)
        vs.append(torch.ones(len(i)))
        i2.append(i)
        i1.append(torch.tensor([idx] * len(i)))
    
    i = torch.stack([torch.cat(i1, dim=0), torch.cat(i2, dim=0)], dim=0)
    v = torch.cat(vs, dim=0)
    return torch.sparse_coo_tensor(i, v, mat.size())


def extract_diagonal_matrix(mat):
    size = mat.size()[0]
    d = [mat[i,i] for i in range(size)];
    i = torch.stack([torch.arange(len(d)), torch.arange(len(d))], dim=0)
    v = torch.tensor(d)
    return torch.sparse_coo_tensor(i, v, mat.size())


In [1]:
import torch

In [6]:
A = torch.arange(9).view(3,3)

In [7]:
A[torch.tensor([[0,1], [1, 2]])]

tensor([[[0, 1, 2],
         [3, 4, 5]],

        [[3, 4, 5],
         [6, 7, 8]]])

In [8]:
l =[[0,1], [1, 2]]

In [14]:
A[0, torch.tensor([0,2])]

tensor([0, 2])